### Load data set with features

In [1]:
from create_df import read_df
df, dfc, all_homes, appliance_min, national_average = read_df()

Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 29 days


In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


In [3]:
run all_functions.py

In [4]:
run features.py

In [5]:
import json

In [8]:
appliances = ["hvac","dr", "light","fridge","wm","dw"]
out = {}
import json
for appliance in appliances:
    out[appliance]={}
    for feature in ["Monthly"]:
        print appliance, feature
        out[appliance]={}
        o = json.load(open("../sensitivity-data/%s_%s.json" %(appliance, feature),'r'))
        f = o[appliance][feature]['f']
        k = o[appliance][feature]['k']
        temp= create_predictions(df, dfc, all_homes, appliance_min, 
                                              national_average, appliance=appliance,
                                                feature=f, NUM_NEIGHBOURS=k,
                                                   train_outlier=True, test_outlier=False, 
                                                outlier_features=None,
                                                outlier_fraction=0.1)
        errors = {}
        if appliance =="hvac":
            start_month, end_month = 5, 11
        else:
            start_month, end_month = 1, 13
        for i in range(start_month, end_month):
            errors[i] = percentage_error(temp[i]["gt"], temp[i]["pred"])
            error_df = pd.DataFrame(errors)
            accur_df = 100-error_df
            accur_df[accur_df<0]=0

        tdf = accur_df
        if appliance =="hvac":
            for home in [624, 1953, 6636, 6836, 7769, 9922]:
                tdf.loc[home, 5]=np.NaN
                tdf.loc[home, 10]=np.NaN

        out[appliance]=tdf
            
            

hvac Monthly
dr Monthly
light Monthly
fridge Monthly
wm Monthly
dw Monthly


In [21]:
p = pd.Panel(out)

In [22]:
p

<class 'pandas.core.panel.Panel'>
Dimensions: 6 (items) x 39 (major_axis) x 12 (minor_axis)
Items axis: dr to wm
Major_axis axis: 26 to 9982
Minor_axis axis: 1 to 12

In [14]:
out["fridge"].mean().mean()

77.740659617682553

In [15]:
accuracy_store = pd.HDFStore("accuracy.h5")

In [16]:
accuracy_store.remove('N-NILM')

In [17]:
accuracy_store.append("N-NILM", p)

/Users/nipunbatra/anaconda/lib/python2.7/site-packages/tables/path.py:100: NaturalNameWarning: object name is not a valid Python identifier: 'N-NILM'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  NaturalNameWarning)


In [18]:
accuracy_store.close()